# Display morphology population features

Copyright (c) 2025 Open Brain Institute

Authors: Michael W. Reimann

last modified: 08.2025

## Summary
This notebook analyzes a list of neuron morphology files. It extracts neurite or morphology features from them and displays the distribution of their values as a histogram. 
For details, see the [README](README.md).

## Imports and setting up platform authentication

Please follow the displayed instructions to authenticate.

In [ ]:
from obi_auth import get_token
from entitysdk.client import Client
from entitysdk.models import ReconstructionMorphology
import pandas as pd

import neurom
import os
import tqdm

from ipywidgets import widgets
from neurom.check.runner import CheckRunner
from matplotlib import pyplot as plt
from obi_notebook import get_projects
from obi_notebook import get_entities


## Selection of morphologies and download

Morphologies will be downloaded to `analysis_morphologies`, then loaded and analyzed

#### Project selection
As a first step we select one of the projects we have access to that the morphologies are associated with. If the morphologies of interest are part of the public OBI assets, any project can be selected.

In [ ]:
token = get_token(environment="production", auth_mode="daf")
project_context = get_projects.get_projects(token)

#### Morphology selection

Next, we select the morphologies. If you already know the unique identifiers of the morphologies of interest, paste them below as a list (!) into line 4 of the next cell.

Otherwise, a widget for morphology selection will be created that allows you to filter and select multipe morphologies.

In [ ]:
client = Client(environment="production", project_context=project_context, token_manager=token)

# Optional: Download using unique ID
entity_IDs = "<MORPHOLOGY-IDs>"  # <<< FILL IN A LIST OF UNIQUE MORPHOLOGY IDENTIFIERS HERE


if entity_IDs != "<MORPHOLOGY-IDs>":
    morph_ids = [entity_IDs]
else:
# Alternative: Select from a table of entities
    morph_ids = []
    morph_ids = get_entities.get_entities("reconstruction-morphology", token, morph_ids,
                                            project_context=project_context,
                                            multi_select=True,
                                            page_size=100)

#### Selection of morphology file type

Moprphologies can be stored in different file formats. While in principle conversion between the formats can be done with no (or minimal) loss, in some cases differences can still occur. Therefore, we have to explicitly select which of the available formats to use in the widget below.

The numbers after the file format indicate how many of the selected neuron morphologies are available in each format. If a format is not available for a selected morphology, it will be skipped.

In [ ]:
fetched = [client.get_entity(entity_id=morph_id_, entity_type=ReconstructionMorphology) for morph_id_ in morph_ids]

asset_types = {}
for fetched_ in fetched:
    for asset in fetched_.assets:
        tp_name = str(asset.content_type)
        asset_types[tp_name] = asset_types.setdefault(tp_name, 0) + 1

asset_type_str = [(f"{k}: {v} / {len(fetched)}", k) for k, v, in asset_types.items()]

asset_type_sel = widgets.Dropdown(options=asset_type_str)
display(asset_type_sel)

#### Fetch morphologies
The selected morphology files are copied to the local system and loaded.

In [ ]:
all_assets = [[asset for asset in fetched_.assets if str(asset.content_type)==asset_type_sel.value]
              for fetched_ in fetched]

morphologies_root = "analysis_morphologies"
if not os.path.exists(morphologies_root):
    os.makedirs(morphologies_root)

nm_morphs = {}
for asset_, morph_id_, fetched_ in tqdm.tqdm(list(zip(all_assets, morph_ids, fetched))):
    if len(asset_) >= 1:
        out_fn = client.download_file(
            entity_id=morph_id_,
            entity_type=ReconstructionMorphology,
            asset_id=asset_[0],
            output_path=morphologies_root,
            project_context=project_context
        )
        nm_morphs.setdefault("_morphology", []).append(neurom.load_morphology(out_fn))
        nm_morphs.setdefault("Region", []).append(fetched_.brain_region.acronym)
        nm_morphs.setdefault("Morphology type", []).append(fetched_.mtypes[0].pref_label)
        nm_morphs.setdefault("Created by", []).append(fetched_.created_by.pref_label)
nm_morphs = pd.DataFrame(nm_morphs)


Helper functions for analysis and plotting

In [ ]:
import pandas, numpy

def lookup(tp_feature, str_feature):
    def inner_func(obj, row, **kwargs):
        vals = neurom.features.get(str_feature, obj)
        if not hasattr(vals, "__iter__"):
                vals = [vals]
        ret = pandas.DataFrame({"value": vals})
        for idx in row.index:
            if not idx.startswith("_"): ret[idx] = row[idx]
        for k, v in kwargs.items(): ret[k] = v
        return ret
    
    def func(row):
        m = row["_morphology"]
        if tp_feature == 1:
            return inner_func(m, row)
        out = []
        for _nrt in m.neurites:
            out.append(inner_func(_nrt, row, neurite_type=str(_nrt.type)))
        return pandas.concat(out, axis=0)
    return func

def histogram_series(data, **kwargs):
    vals, bin_c = numpy.histogram(data, **kwargs)
    return pandas.Series(vals, index=0.5 * (bin_c[:-1] + bin_c[1:]))

Select the morphology feature to display from the dropdown menu.

In [ ]:
function = widgets.Dropdown(
    options=
    [(k, (0, k)) for k, v in neurom.features._NEURITE_FEATURES.items()] +
    [(k, (1, k)) for k, v in neurom.features._MORPHOLOGY_FEATURES.items()], 
    description='Feature')
display(function)

Select from the dropdown menus by which parameter to group the data (or "None" for no grouping).

Select with the slider the number of bins to show.

In [ ]:
data_df = pandas.concat(nm_morphs.apply(lookup(*function.value), axis=1).values, axis=0)
data_df["None"] = "None"
grouper = widgets.Dropdown(
    options=[_col for _col in data_df.columns if _col != "value"],
    description="Grouped by"
)
nbins = widgets.IntSlider(10, 5, 100, 1)
display(grouper)
display(nbins)

In [ ]:
span = data_df["value"].max() - data_df["value"].min()
bins = numpy.linspace(data_df["value"].min(), data_df["value"].max() + 1E-9 * span, nbins.value + 1)
w = numpy.mean(numpy.diff(bins)) * 0.8
hist_df = data_df.groupby(grouper.value)["value"].apply(histogram_series, bins=bins).unstack(grouper.value)

fig = plt.figure(figsize=(6, 3))
ax = fig.gca()
ax.set_frame_on(False)

bot = numpy.zeros(len(hist_df))
for col in hist_df.columns:
    ax.bar(hist_df.index, hist_df[col].values, bottom=bot, label=col, width=w)
    bot += hist_df[col].values
ax.set_xlabel(function.value[1])
ax.set_ylabel("Count")
if len(hist_df.columns) < 10:
    plt.legend()